In [1]:
from fastai.vision import *
import torch
torch.cuda.set_device(0)

In [6]:
path = untar_data(URLs.IMAGENETTE)

In [40]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path, train = 'train', valid = 'val', bs = 64, size = 224, ds_tfms = tfms).normalize(imagenet_stats)

In [ ]:
data.show_batch()

In [44]:
learn = cnn_learner(data, models.resnet34, metrics = accuracy, pretrained = True)
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 112, 112]       9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 112, 112]       128        True      
______________________________________________________________________
ReLU                 [64, 112, 112]       0          False     
______________________________________________________________________
MaxPool2d            [64, 56, 56]         0          False     
______________________________________________________________________
Conv2d               [64, 56, 56]         36,864     False     
______________________________________________________________________
BatchNorm2d          [64, 56, 56]         128        True      
______________________________________________________________________
ReLU                 [64, 56, 56]         0          False     
___________________________________________________

In [ ]:
learn.fit_one_cycle(10, max_lr = 1e-2)

In [46]:
learn.save('imagenet_bs64')